In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Network Structure

In [20]:
class network_structure():
    def __init__(self, edges_nodes_matrix):
        # Structure of edges_nodes_matrix
        # An array of 2d-arrays, with each 2d-array encoding the start and end node indices of an edge.
        
        assert edges_nodes_matrix.shape[1] == 2, "We must have edges_nodes_matrix.shape[1] == 2"
        
        self.edges_nodes_matrix = edges_nodes_matrix
        self.nodes = np.unique(self.edges_nodes_matrix.flatten())
        self.edges = np.array(range(self.edges_nodes_matrix.shape[0]))
        self.num_nodes = len(self.nodes)
        self.num_edges = len(self.edges)
        
        self.construct_nodes_dict()
        self.construct_edges_dict()
#         self.construct_depth_dicts()
#         self.construct_height_dicts()
    
        origin_nodes = [node for node in self.nodes if self.nodes_dict[node]['incoming_edges'] == []]
        destination_nodes = [node for node in self.nodes if self.nodes_dict[node]['outgoing_edges'] == []]
        
        assert len(origin_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        assert len(destination_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        self.origin_node = origin_nodes[0]
        self.destination_node = destination_nodes[0]
        self.intermediate_nodes = [node for node in self.nodes if node not in [self.origin_node, self.destination_node]]
        
        self.start_edges = self.nodes_dict[self.origin_node]["outgoing_edges"]
        self.end_edges = self.nodes_dict[self.destination_node]["incoming_edges"]
        
        self.construct_routes_dict()
    
        return

    
    # Network structure functions:
    
    def construct_nodes_dict(self):
        self.nodes_dict = {}
        for node in self.nodes:
            self.nodes_dict[node] = {}
            self.nodes_dict[node]['incoming_edges'] = list((self.edges_nodes_matrix[:, 1] == node).nonzero()[0])
            self.nodes_dict[node]['outgoing_edges'] = list((self.edges_nodes_matrix[:, 0] == node).nonzero()[0])
            
            self.nodes_dict[node]['previous_nodes'] = list(set(self.edges_nodes_matrix[edge, 0] for edge in self.nodes_dict[node]['incoming_edges']))
            self.nodes_dict[node]['next_nodes'] = list(set(self.edges_nodes_matrix[edge, 1] for edge in self.nodes_dict[node]['outgoing_edges']))
        
        return

    def construct_edges_dict(self):
        self.edges_dict = {}

        for edge in self.edges:
            self.edges_dict[edge] = {}

        for node in self.nodes:
            if self.nodes_dict[node]['incoming_edges'] != []:
                for edge in self.nodes_dict[node]['incoming_edges']:
                    self.edges_dict[edge]['end_node'] = node

            if self.nodes_dict[node]['outgoing_edges'] != []:
                for edge in self.nodes_dict[node]['outgoing_edges']:
                    self.edges_dict[edge]['start_node'] = node

        for edge in self.edges:
            self.edges_dict[edge] = {}
            start_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['outgoing_edges']]))
            end_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['incoming_edges']]))
            self.edges_dict[edge]['start_node'] = start_node
            self.edges_dict[edge]['end_node'] = end_node
            self.edges_dict[edge]['incoming_edges'] = self.nodes_dict[start_node]['incoming_edges']
            self.edges_dict[edge]['outgoing_edges'] = self.nodes_dict[end_node]['outgoing_edges']
        return
    
    def construct_routes_dict(self):
        self.routes = [[edge] for edge in self.start_edges]
        
        route_construction_finished = False
        while route_construction_finished is False:
            route_construction_finished = True
            routes_temp = []
            for route_segment in self.routes:
                
#                 print()
#                 print("route_segment:", route_segment)
#                 print("route_segment[-1]:", route_segment[-1])
#                 print("self.edges_dict[route_segment[-1]]:", self.edges_dict[route_segment[-1]])
#                 print("self.edges_dict[route_segment[-1]][outgoing_edges]:", self.edges_dict[route_segment[-1]]["outgoing_edges"])
# #                 print()
                
                if self.edges_dict[route_segment[-1]]["outgoing_edges"] != []:
                    route_construction_finished = False
                    for edge in self.edges_dict[route_segment[-1]]["outgoing_edges"]:
                        routes_temp.append(route_segment + [edge])
                else:
                    routes_temp.append(route_segment)
                
#                 print("routes_temp:", routes_temp)
                
                
            if route_construction_finished is True:
                break
            self.routes = routes_temp
            
#             print()
#             print("self.routes:", self.routes)
#             print()

        return
    
    def print_nodes_dict(self):
        print()
        print("Printing nodes:")
        for node_key, node_value in self.nodes_dict.items():
            print()
            print("Node", node_key)
            for key, value in node_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_edges_dict(self):
        print()
        print("Printing edges:")
        for edge_key, edge_value in self.edges_dict.items():
            print()
            print("Edge", edge_key)
            for key, value in edge_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_routes(self):
        print()
        print("Printing routes:")
        print()
        for route_index, route_edges in enumerate(self.routes):
            print("Route", route_index, ":", route_edges)
        return
    
    def print_all(self):
        print()
        print("Printing network information:")
        print()
        print("Origin node:", self.origin_node)
        print("Destination node:", self.destination_node)
        print("Number of nodes:", self.num_nodes)
        print()
        print("Starting edges:", self.start_edges)
        print("Ending edges:", self.end_edges)
        print("Number of edges:", self.num_edges)
        
        self.print_nodes_dict()
        self.print_edges_dict()
        self.print_routes()
        
        return
    

In [21]:
# list_1 = [1, 2]
# list_2 = [3, 4, 5]
# list_1 = list_1 + [6]
# list_2.append(list_1)
# list_2

In [22]:
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
netw_example = network_structure(edges_nodes_matrix)

# print("netw_example.edges:", netw_example.edges)
# print()
# print("netw_example.nodes:", netw_example.nodes)
# print()
# print("netw_example.edges_dict:", netw_example.edges_dict)
# print()
# print("netw_example.nodes_dict:", netw_example.nodes_dict)
# print()
# print("netw_example.routes:", netw_example.routes)
# print()

# netw_example.print_nodes_dict()
# netw_example.print_edges_dict()
# netw_example.print_routes()

netw_example.print_all()
# netw_example.intermediate_nodes



Printing network information:

Origin node: 0
Destination node: 3
Number of nodes: 4

Starting edges: [0, 1]
Ending edges: [3, 4]
Number of edges: 5

Printing nodes:

Node 0
incoming_edges : []
outgoing_edges : [0, 1]
previous_nodes : []
next_nodes : [1, 2]

Node 1
incoming_edges : [0]
outgoing_edges : [2, 3]
previous_nodes : [0]
next_nodes : [2, 3]

Node 2
incoming_edges : [1, 2]
outgoing_edges : [4]
previous_nodes : [0, 1]
next_nodes : [3]

Node 3
incoming_edges : [3, 4]
outgoing_edges : []
previous_nodes : [1, 2]
next_nodes : []


Printing edges:

Edge 0
start_node : 0
end_node : 1
incoming_edges : []
outgoing_edges : [2, 3]

Edge 1
start_node : 0
end_node : 2
incoming_edges : []
outgoing_edges : [4]

Edge 2
start_node : 1
end_node : 2
incoming_edges : [0]
outgoing_edges : [4]

Edge 3
start_node : 1
end_node : 3
incoming_edges : [0]
outgoing_edges : []

Edge 4
start_node : 2
end_node : 3
incoming_edges : [1, 2]
outgoing_edges : []


Printing routes:

Route 0 : [0, 2, 4]
Route 1 : [

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [23]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [97]:
def welfare_obj(network, lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_ex[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
        
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = a_ex[4] * (x_ex[e] ** 4) + a_ex[3] * (x_ex[e] ** 3) + a_ex[2] * (x_ex[e] ** 2) \
                    + a_ex[1] * x_ex[e] + a_ex[0]
        ell_gp[e] = a_gp[4] * (x_gp[e] ** 4) + a_gp[3] * (x_gp[e] ** 3) + a_gp[2] * (x_gp[e] ** 2) \
                    + a_gp[1] * x_gp[e] + a_gp[0]
    
    obj_E = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el) for e in range(num_edges)) \
                + sum(y_el[(g, e, k)] * v_E_array[g] * ell_ex[e] for g in range(num_el) for k in [0, 1] for e in range(num_edges)) \
                + sum(y_el[(g, e, 2)] * v_E_array[g] * ell_gp[e] for g in range(num_el) for e in range(num_edges))
    obj_I = sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in) for e in range(num_edges)) \
                + sum(y_in[(g, e, 0)] * v_I_array[g] * ell_ex[e] for g in range(num_in) for e in range(num_edges)) \
                + sum(y_in[(g, e, 1)] * v_I_array[g] * ell_gp[e] for g in range(num_in) for e in range(num_edges))
    obj_R = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el) for e in range(num_edges)) \
                + sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in) for e in range(num_edges))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare

    

In [102]:
def proj_tau_B_11(network, tau, B, tau_max = 1.0, B_max = 1.0):
    
    num_edges = len(network.edges)
    num_nodes = len(network.nodes)
    
        
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()
    
    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    tau_feas = cp.Variable(num_edges)
    B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

    objective = cp.Minimize(func)
        
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max * np.ones(num_edges)]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= sum(tau_feas[edge] for edge in route) for route in network.routes]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(tau_feas.value, decimals=4), np.round(B_feas.value, decimals=4)


# Chinmay's Algorithm:

In [103]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [106]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(network, tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el[(g, e, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in[(g, e, k)] = cp.Variable(1)

    x_ex = cp.Variable(network.num_edges)
    x_gp = cp.Variable(network.num_edges)
    
    # Objective:
    func = 0.0
    for e in network.edges:
        func += 1/5 * a_ex[4] * cp.power(x_ex[e], 5)
        func += 1/4 * a_ex[3] * cp.power(x_ex[e], 4)
        func += 1/3 * a_ex[2] * cp.power(x_ex[e], 3)
        func += 1/2 * a_ex[1] * cp.power(x_ex[e], 2)
        func += a_ex[0] * x_ex[e]
        func += 1/5 * a_gp[4] * cp.power(x_gp[e], 5)
        func += 1/4 * a_gp[3] * cp.power(x_gp[e], 4)
        func += 1/3 * a_gp[2] * cp.power(x_gp[e], 3)
        func += 1/2 * a_gp[1] * cp.power(x_gp[e], 2)
        func += a_gp[0] * x_gp[e]
        
        func += sum(tau[e] * y_el[(g, e, 1)] / v_E_array[g] for g in range(num_el))
        func += sum(tau[e] * y_in[(g, e, 0)] / v_I_array[g] for g in range(num_in))

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, e, k)] >= 0.0 for g in range(num_el) for e in network.edges for k in [0, 1, 2]]
    constraints += [y_in[(g, e, k)] >= 0.0 for g in range(num_in) for e in network.edges for k in [0, 1]]
        
    constraints += [sum(y_el[(g, e, k)] for e in network.start_edges for k in [0, 1, 2]) == 1.0 \
                    for g in range(num_el)]
    constraints += [sum(y_in[(g, e, k)] for e in network.start_edges for k in [0, 1]) == 1.0 \
                    for g in range(num_in)]
    for node in network.intermediate_nodes:
        constraints += [sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1, 2]) \
                        == sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1, 2]) \
                        for g in range(num_el)]
        constraints += [sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1]) \
                        == sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1]) \
                        for g in range(num_in)]
    
    for e in network.edges:
        constraints += [x_ex[e] == sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1]) \
                                   + sum(y_in[(g, e, 0)] for g in range(num_in)) ]
        constraints += [x_gp[e] == sum(y_el[(g, e, 2)] for g in range(num_el)) \
                                   + sum(y_in[(g, e, 1)] for g in range(num_in)) ]
    
    constraints += [sum(y_el[(g, e, 0)]  * tau[e] for e in network.edges) <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el_values[(g, e, k)] = y_el[(g, e, k)].value[0]

    y_in_values = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in_values[(g, e, k)] = y_in[(g, e, k)].value[0]

    return y_el_values, y_in_values


In [108]:
time_1 = time.time()

edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
network_Braess = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

network = network_Braess

## TODO: Edit below to use the fact that creatively, CVXPY can handle multiple-dimensional n ,\
## (e.g., 3d or 4d) arrays:\n,

tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4])
B = 0.8
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 2.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
tau_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros((network.num_edges, num_iters))
tau_perturbed = np.zeros((network.num_edges, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[:, 0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[:, 0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[:, 0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(network.num_edges +1)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[:, i] = tau[:, i] + delta[i] * w_i[:-1]
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, i] < B_perturbed[i] or tau_perturbed[:, i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[:, i], B_perturbed[i] = proj_tau_B_11(network, tau_perturbed[:, i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
#     print("tau[:, i]:", tau[:, i])
#     print("B[i]:", B[i])
#     print("tau_perturbed[:, i]:", tau_perturbed[:, i])
#     print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(network, tau = tau[:, i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

#     print()
#     print("y_el_values:", y_el_values)
#     print("y_in_values:", y_in_values)
#     print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(network, tau = tau_perturbed[:, i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau[:, i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau_perturbed[:, i], \
                                    v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    welfare_list.append(welfare)
    
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print("tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare):\n", tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare))
    
    tau[:, i+1] = tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare)
    
#     print()
#     print("B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare):\n", \
#          B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare))
#     print("B[i]:", B[i])
#     print("eta[i]:", eta[i])
#     print("delta[i]:", delta[i])
#     print("w_i[-1]:", w_i[-1])
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print()
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    tau[:, i+1], B[i+1] = proj_tau_B_11(network, tau[:, i+1], B[i+1], tau_max, B_max)
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.linalg.norm(tau[:, i-diffs_num_cols : i-1] - tau[:, i-diffs_num_cols+1 : i], axis = 0)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, 0:10]:", tau[:, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [-0.64233544 -0.49843641 -0.25721537 -0.084286   -0.44947943 -0.25233054]

Iter: 1
w_i: [ 0.44874705 -0.28676895 -0.37716929  0.00867178  0.40182688 -0.64233398]

Iter: 2
w_i: [ 0.29584991 -0.10082251  0.34359963 -0.21703714 -0.83737529  0.18959021]

Iter: 3
w_i: [ 0.12458686 -0.54760298 -0.51350577  0.1645151   0.09161211  0.6208566 ]

Iter: 4
w_i: [-0.35460991  0.57790825 -0.20261287 -0.44255178 -0.44588242  0.32335533]

Iter: 5
w_i: [-0.380426   -0.37838044 -0.35498996  0.54016035 -0.02674904  0.54184658]

Iter: 6
w_i: [ 0.14223852  0.09525145 -0.6318269  -0.20845615 -0.40355208  0.60430281]

Iter: 7
w_i: [ 0.23238074 -0.14845989  0.74850544  0.34028199  0.01872441 -0.49755002]
tau_diffs: [6.58321218 6.79160076 8.86619794 8.48528137]
B_diffs: [ 0.  0.  0. -0.]

Iter: 8
w_i: [-0.07733011  0.376062   -0.72730084  0.17880931  0.20133346 -0.50112173]
tau_diffs: [6.79160076 8.86619794 8.48528137 4.85060504]
B_diffs: [ 0.  0. -0.  0.]

Iter: 9
w_i: [ 0.08019406 -0.39869868  

tau_diffs: [10.48532655  5.12416503  7.43056868  1.90494412]
B_diffs: [ 0.2349 -0.      0.      0.    ]

Iter: 49
w_i: [-0.32547276  0.16050251  0.92911177  0.053238    0.02715162  0.03855191]
tau_diffs: [5.12416503 7.43056868 1.90494412 6.29455397]
B_diffs: [-0.      0.      0.     -1.4894]

Iter: 50
w_i: [ 0.32935043 -0.2279807   0.71453045  0.29321312 -0.38275544  0.31068259]
tau_diffs: [7.43056868 1.90494412 6.29455397 1.92384626]
B_diffs: [ 0.      0.     -1.4894  1.4894]

Iter: 51
w_i: [ 0.90411781 -0.29644573  0.11329144  0.25416615  0.10503624  0.07888552]
tau_diffs: [1.90494412 6.29455397 1.92384626 1.06440629]
B_diffs: [ 0.     -1.4894  1.4894  0.    ]

Iter: 52
w_i: [-0.00138002 -0.06259505 -0.4836884   0.33691775 -0.7995332  -0.0967397 ]
tau_diffs: [ 6.29455397  1.92384626  1.06440629 10.25265346]
B_diffs: [-1.4894  1.4894  0.      0.    ]

Iter: 53
w_i: [-0.49949024 -0.15980826 -0.45577391 -0.15064846  0.70265997 -0.02854771]
tau_diffs: [ 1.92384626  1.06440629 10.25265346

tau_diffs: [0.10515798 0.08585319 1.16852061 1.04526707]
B_diffs: [ 0.0325 -0.1778  0.1778  0.    ]

Iter: 93
w_i: [-0.34221142 -0.12642992 -0.23911114  0.11731116 -0.88814306 -0.08469169]
tau_diffs: [0.08585319 1.16852061 1.04526707 1.25087728]
B_diffs: [-0.1778  0.1778  0.     -0.5213]

Iter: 94
w_i: [-0.10093087 -0.88610995 -0.26591889  0.12776352 -0.29705414 -0.17130262]
tau_diffs: [1.16852061 1.04526707 1.25087728 0.64124347]
B_diffs: [ 0.1778  0.     -0.5213  0.5213]

Iter: 95
w_i: [ 0.20684622  0.69682228 -0.41432267 -0.53932135 -0.05525149  0.07790909]
tau_diffs: [1.04526707 1.25087728 0.64124347 0.19771864]
B_diffs: [ 0.     -0.5213  0.5213 -0.0629]

Iter: 96
w_i: [-0.57880471  0.12029817  0.11033019 -0.66764686  0.18393191 -0.39844375]
tau_diffs: [1.25087728 0.64124347 0.19771864 0.09058852]
B_diffs: [-0.5213  0.5213 -0.0629  0.0157]

Iter: 97
w_i: [-0.6534577   0.02505213 -0.33414517  0.47736534  0.08367844 -0.47521855]
tau_diffs: [0.64124347 0.19771864 0.09058852 0.52875951

tau_diffs: [0.13309508 2.57557453 0.12002554 0.20439428]
B_diffs: [-0.2414  0.2414 -0.0336  0.0336]

Iter: 137
w_i: [-0.70933029  0.35184366 -0.38724587  0.39198734  0.19954947 -0.17211378]
tau_diffs: [2.57557453 0.12002554 0.20439428 0.22599644]
B_diffs: [ 0.2414 -0.0336  0.0336  0.    ]

Iter: 138
w_i: [-0.4641363  -0.10764814 -0.26595257 -0.81935346  0.15070732  0.09058601]
tau_diffs: [0.12002554 0.20439428 0.22599644 1.90354928]
B_diffs: [-0.0336  0.0336  0.      0.    ]

Iter: 139
w_i: [-0.14648796 -0.21569462 -0.54342324  0.43973691  0.65371158 -0.12649471]
tau_diffs: [0.20439428 0.22599644 1.90354928 0.36978726]
B_diffs: [0.0336 0.     0.     0.    ]

Iter: 140
w_i: [ 0.35195545  0.08753576 -0.50465499 -0.33348105 -0.3826899   0.59676379]
tau_diffs: [0.22599644 1.90354928 0.36978726 0.42086593]
B_diffs: [ 0.  0.  0. -0.]

Iter: 141
w_i: [ 0.23129065 -0.39506842  0.44650486  0.73459226  0.22547787  0.02435025]
tau_diffs: [1.90354928 0.36978726 0.42086593 0.31983013]
B_diffs: [ 0.

tau_diffs: [1.34040266 3.10675885 3.63884626 0.02166218]
B_diffs: [ 0.0577  0.      0.     -0.0005]

Iter: 182
w_i: [-0.38613042 -0.40407121 -0.30182001  0.31991366 -0.108257    0.69460068]
tau_diffs: [3.10675885 3.63884626 0.02166218 4.79668886]
B_diffs: [ 0.      0.     -0.0005  0.0005]

Iter: 183
w_i: [-0.35406145  0.43210123 -0.59205904 -0.09967737  0.46887695 -0.32804563]
tau_diffs: [3.63884626 0.02166218 4.79668886 0.65128479]
B_diffs: [ 0.     -0.0005  0.0005  0.    ]

Iter: 184
w_i: [ 0.30585691 -0.03880976  0.04039317  0.61744894 -0.63595935  0.34296685]
tau_diffs: [0.02166218 4.79668886 0.65128479 5.75670455]
B_diffs: [-0.0005  0.0005  0.      0.    ]

Iter: 185
w_i: [-0.15038198 -0.4962435   0.72150844  0.1637871   0.40804202  0.13125822]
tau_diffs: [4.79668886 0.65128479 5.75670455 0.24223423]
B_diffs: [ 0.0005  0.      0.     -0.108 ]

Iter: 186
w_i: [-0.11600969 -0.27080321 -0.2231259  -0.82989508  0.37700575  0.18045228]
tau_diffs: [0.65128479 5.75670455 0.24223423 1.392

...

tau_diffs: [0.00931075 0.24307795 0.2077006  0.68103846]
B_diffs: [ 0.    -0.078  0.078  0.   ]

Iter: 801
w_i: [ 0.41668956  0.53044253 -0.31665308  0.26409894  0.20297871  0.57773935]
tau_diffs: [0.24307795 0.2077006  0.68103846 0.36076133]
B_diffs: [-0.078  0.078  0.     0.   ]

Iter: 802
w_i: [ 0.20018619 -0.44295108 -0.20570298 -0.42694209 -0.66572124  0.30974473]
tau_diffs: [0.2077006  0.68103846 0.36076133 0.13857258]
B_diffs: [ 0.078  0.     0.    -0.155]

Iter: 803
w_i: [ 0.4346523   0.02965617  0.39553397  0.45748949  0.27699267 -0.60640679]
tau_diffs: [0.68103846 0.36076133 0.13857258 0.79819328]
B_diffs: [ 0.     0.    -0.155  0.155]

Iter: 804
w_i: [-0.21488597  0.15747893 -0.11614853 -0.56662582 -0.1532657   0.75563135]
tau_diffs: [0.36076133 0.13857258 0.79819328 0.75947651]
B_diffs: [ 0.    -0.155  0.155  0.   ]

Iter: 805
w_i: [ 0.20821798  0.37757935  0.40623682 -0.70993251 -0.27560089 -0.26285115]
tau_diffs: [0.13857258 0.79819328 0.75947651 0.05562994]
B_diffs: [-0.

tau_diffs: [0.38949688 0.96406596 0.74283683 0.64898708]
B_diffs: [-0.  0.  0. -0.]

Iter: 846
w_i: [ 0.08958875 -0.84452078  0.49591377 -0.03738866 -0.17723741  0.00412698]
tau_diffs: [0.96406596 0.74283683 0.64898708 0.07548669]
B_diffs: [ 0.  0. -0.  0.]

Iter: 847
w_i: [ 0.13353764  0.65488976  0.06119162 -0.59321924 -0.43339406 -0.09901121]
tau_diffs: [0.74283683 0.64898708 0.07548669 1.61320479]
B_diffs: [ 0. -0.  0.  0.]

Iter: 848
w_i: [ 0.80207001 -0.38435415  0.24409876  0.02848498 -0.27619552 -0.26884198]
tau_diffs: [0.64898708 0.07548669 1.61320479 0.10417082]
B_diffs: [-0.  0.  0.  0.]

Iter: 849
w_i: [ 0.70950842 -0.13597699 -0.01728215  0.52132658 -0.44628905  0.08278933]
tau_diffs: [0.07548669 1.61320479 0.10417082 0.18084518]
B_diffs: [ 0.     0.     0.    -0.018]

Iter: 850
w_i: [-0.42079979  0.18771224  0.05875034 -0.05385565  0.3544939  -0.8097368 ]
tau_diffs: [1.61320479 0.10417082 0.18084518 0.15131048]
B_diffs: [ 0.     0.    -0.018  0.018]

Iter: 851
w_i: [ 0.26

tau_diffs: [0.78065214 0.12955451 0.88775526 1.17983969]
B_diffs: [ 0.     -0.0568  0.0568 -0.    ]

Iter: 890
w_i: [ 0.49032628 -0.05730608  0.58007969  0.23921365 -0.38623446  0.46195614]
tau_diffs: [0.12955451 0.88775526 1.17983969 0.10202436]
B_diffs: [-0.0568  0.0568 -0.      0.    ]

Iter: 891
w_i: [-0.30011728  0.84371649 -0.36202923  0.1376704   0.20632854 -0.07404278]
tau_diffs: [0.88775526 1.17983969 0.10202436 0.03955831]
B_diffs: [ 0.0568 -0.      0.      0.    ]

Iter: 892
w_i: [-0.1433239  -0.74662513  0.32148664  0.29968217 -0.39967353  0.26288243]
tau_diffs: [1.17983969 0.10202436 0.03955831 1.07248554]
B_diffs: [-0.  0.  0.  0.]

Iter: 893
w_i: [-0.09978481 -0.25305408 -0.24300332 -0.36053289 -0.83683733  0.19150803]
tau_diffs: [0.10202436 0.03955831 1.07248554 0.02437253]
B_diffs: [0. 0. 0. 0.]

Iter: 894
w_i: [ 0.25758584 -0.30326512  0.26300583  0.70755091 -0.42174882 -0.30660623]
tau_diffs: [0.03955831 1.07248554 0.02437253 0.81680709]
B_diffs: [0. 0. 0. 0.]

Iter:

tau_diffs: [1.05331461 0.01340261 1.19872605 0.12959749]
B_diffs: [ 0.     -0.0021  0.0021 -0.    ]

Iter: 936
w_i: [-0.3664198  -0.2669522  -0.31319943  0.54636686 -0.62402713 -0.09193783]
tau_diffs: [0.01340261 1.19872605 0.12959749 0.35059528]
B_diffs: [-0.0021  0.0021 -0.      0.    ]

Iter: 937
w_i: [ 0.49227909  0.84723303  0.08087138 -0.08038585  0.13675718 -0.09029339]
tau_diffs: [1.19872605 0.12959749 0.35059528 0.17992562]
B_diffs: [ 0.0021 -0.      0.      0.    ]

Iter: 938
w_i: [ 0.10533782 -0.33063069 -0.31506652 -0.15586138  0.82861098  0.26349847]
tau_diffs: [0.12959749 0.35059528 0.17992562 0.32332383]
B_diffs: [-0.      0.      0.     -0.0298]

Iter: 939
w_i: [-0.33867939 -0.27503045 -0.60007415  0.56886851 -0.00927076  0.35477909]
tau_diffs: [0.35059528 0.17992562 0.32332383 0.26124751]
B_diffs: [ 0.      0.     -0.0298  0.0298]

Iter: 940
w_i: [-0.18265822 -0.33853111 -0.65382176  0.52563099  0.21429393  0.31990615]
tau_diffs: [0.17992562 0.32332383 0.26124751 0.505

tau_diffs: [0.17329795 0.31710418 0.03931603 0.74787811]
B_diffs: [0.0096 0.     0.     0.    ]

Iter: 980
w_i: [-0.60781809  0.26601326 -0.19396436  0.40773891  0.44163421  0.40109867]
tau_diffs: [0.31710418 0.03931603 0.74787811 0.11136503]
B_diffs: [ 0.      0.      0.     -0.0102]

Iter: 981
w_i: [0.04096252 0.47246394 0.2199168  0.63732153 0.56481612 0.03924952]
tau_diffs: [0.03931603 0.74787811 0.11136503 0.08529555]
B_diffs: [ 0.      0.     -0.0102 -0.0728]

Iter: 982
w_i: [ 0.0139159   0.02968986 -0.22401363  0.4969996  -0.74200118  0.3888038 ]
tau_diffs: [0.74787811 0.11136503 0.08529555 0.78328832]
B_diffs: [ 0.     -0.0102 -0.0728  0.083 ]

Iter: 983
w_i: [-0.39839365 -0.23494841  0.03163773 -0.1210925   0.48959606 -0.72850059]
tau_diffs: [0.11136503 0.08529555 0.78328832 0.26957619]
B_diffs: [-0.0102 -0.0728  0.083  -0.0106]

Iter: 984
w_i: [-0.02319048 -0.26200272 -0.76848107  0.36817199  0.45238069 -0.00739716]
tau_diffs: [0.08529555 0.78328832 0.26957619 0.72947128]
B_d

In [109]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

first(welfare_list): 602.6393276262938
min(welfare_list): 537.3057792279053
max(welfare_list): 677.3338012646757
argmin_tau: [6.     6.     0.7824 6.     6.    ]
argmin_B: -0.0


## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Unused Functions for Network:

In [ ]:

def construct_depth_dicts(self):
    self.depth_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['incoming_edges']) == 0]
    self.depth_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['incoming_edges']).issubset(set(edges_accounted))]
        self.depth_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.depths = list(self.depth_edges_dict.keys())

    self.depth_nodes_dict = {}
    for depth in self.depths:
        nodes_temp = np.array([self.edges_dict[edge]['start_node'] for edge in self.depth_edges_dict[depth]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.depth_nodes_dict[depth] = nodes_temp

    return

def construct_height_dicts(self):
    self.height_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['outgoing_edges']) == 0]
    self.height_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['outgoing_edges']).issubset(set(edges_accounted))]
        self.height_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.heights = list(self.height_edges_dict.keys())

    self.height_nodes_dict = {}
    for height in self.heights:
        nodes_temp = np.array([self.edges_dict[edge]['end_node'] for edge in self.height_edges_dict[height]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.height_nodes_dict[height] = nodes_temp

    return

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
